In [2]:
import os

def do_it(folder_id, impl, impl2, features, prefix, is_gpu=False):
    if(is_gpu==False):
        filename= prefix+'/'+'synthetic'+str(folder_id)+'/'+impl+'_d.csv'
    else:
        filename= prefix+'/'+'synthetic'+str(folder_id)+'/'+impl+'.csv'
    if(os.path.exists(filename)):
        with open(filename) as f:
            results = f.readlines()

        if(is_gpu==False):
            filename_new = prefix+'/'+'synthetic_new'+str(folder_id)+'/'+impl+'_d.csv'
        else:
            filename_new = prefix+'/'+'synthetic_new'+str(folder_id)+'/'+impl+'.csv'
        f = open(filename_new,'w')
        if(len(features) == len(results)):
            for i,j in zip(features, results):
                j_stripped = j.split(','+impl2)[1]
                new_line = i.strip('\n') + ',' + impl2 + j_stripped
                f.write(new_line)
        else:
            print(filename, 'has different length than features', '( features =', len(features) , 'vs results =', len(results), ')')
            print('working on it ...')
            cnt = 0
            for j in results:
                j_spl = j.split(',')
                j_spl_new = ','.join(j_spl[0:5])
                
                i = features[cnt]
                i_spl = i.split(',')
                i_spl_new = ','.join(i_spl[0:5])
                
                while(i_spl_new != j_spl_new):
                    cnt+=1
                    i = features[cnt]
                    i_spl = i.split(',')
                    i_spl_new = ','.join(i_spl[0:5])

                if(i_spl_new == j_spl_new):
                    j_stripped = j.split(','+impl2)[1]
                    new_line = i.strip('\n') + ',' + impl2 + j_stripped
                    f.write(new_line)
                cnt+=1
        f.close()
        # print('\t',filename_new, 'done!')
    else:
        print(filename, 'does not exist')


In [3]:
def concat_it(folder_ids, impl_list, impl_list2, prefix, is_gpu=False):
    if(is_gpu==False):
        filenames=[ prefix+'/'+'synthetic_new'+str(folder_id)+'/'+impl+'_d.csv' for folder_id in folder_ids for impl in impl_list]
    else:
        filenames=[ prefix+'/'+'synthetic_new'+str(folder_id)+'/'+impl+'.csv' for folder_id in folder_ids for impl in impl_list]
    filenames = [filename if(os.path.exists(filename)) else '' for filename in filenames]
    filenames = [i for i in filenames if i]
    
    extended_results = prefix + 'results.csv'
    with open(extended_results, 'w') as outfile:
        for fname in filenames:
            with open(fname) as infile:
                for line in infile:
                    outfile.write(line)

In [4]:
features_filename = 'features.txt'
with open(features_filename) as f:
    features = f.readlines()

features_small = []
fw = open('features_small.csv','w')
for i in features:
    i_spl = i.split(',')
    new_i = ",".join(i_spl[0:12]+i_spl[14:16]+i_spl[18:20]+i_spl[22:24]+i_spl[26:28]+i_spl[30:32]+[i_spl[35]])
    features_small.append(new_i)
    fw.write(new_i+'\n')
fw.close()

---

In [ ]:
# with open('features_friends_gold2.csv') as f:
#     features_small2 = f.readlines()
# for i in range(len(features_small2)):
#     features_small2[i] = features_small2[i].strip('\n')
# # print(features_small[0])
# # print(features_small2[0])

# prefix = 'intel-gold2'

# impl_list         = ['csr5', 'csr_naive',     'merge', 'mkl_ie', 'sparsex', 'sell_C_s' ]
# impl_list2        = ['CSR5', 'Naive_CSR_CPU', 'MERGE', 'MKL_IE', 'SparseX', 'SELL-32-1']
# folder_ids        = [1,2,3,4,5]

# for folder_id in folder_ids:
#     for impl,impl2 in zip(impl_list, impl_list2):
#         do_it(folder_id, impl, impl2, features_small2, prefix)

# concat_it(folder_ids, impl_list, impl_list2, prefix)

---

---

In [ ]:
prefix = 'amd_epyc1'

impl_list         = ['aocl_optmv', 'csr5', 'csr_naive',     'merge', 'mkl_ie', 'sparsex', 'sell_C_s' ]
impl_list2        = ['AOCL_OPTMV', 'CSR5', 'Naive_CSR_CPU', 'MERGE', 'MKL_IE', 'SparseX', 'SELL-32-1']
folder_ids        = [1,2,3,4,5]

for folder_id in folder_ids:
    for impl,impl2 in zip(impl_list, impl_list2):
        do_it(folder_id, impl, impl2, features, prefix)

concat_it(folder_ids, impl_list, impl_list2, prefix)

In [ ]:
with open('features_validation_friends2.csv') as f:
    features_small2 = f.readlines()
for i in range(len(features_small2)):
    features_small2[i] = features_small2[i].strip('\n')
# print(features_small[0])
# print(features_small2[0])

prefix = 'power9'

impl_list         = ['xlc_csr_naive', 'xlc_csr',      'merge', 'sparsex']
impl_list2        = ['Naive_CSR_CPU', 'Custom_CSR_B', 'MERGE', 'SparseX']
folder_ids        = [1]

for folder_id in folder_ids:
    for impl,impl2 in zip(impl_list, impl_list2):
        do_it(folder_id, impl, impl2, features_small2, prefix)

concat_it(folder_ids, impl_list, impl_list2, prefix)

In [ ]:
def convert_it(file_old):
    file_good = file_old.replace('.csv', '_GOOD.csv')
    file_new = file_old.replace('.csv', '_NEW.csv')
    fw = open(file_new,'w')
    with open(file_old) as f1:
        with open(file_good) as f2:
            for line1,line2 in zip(f1,f2):
                line1_spl = line1.split(',')
                line2_spl = line2.split(',')
                time = line1_spl[24]
                watt = line2_spl[26]
                line1_spl[26] = line2_spl[26]
                line1_spl[27] = str(round(float(time)*float(watt),5))+'\n'
                line_new = ",".join(line1_spl)
                fw.write(line_new)
    fw.close()

# convert_it('amd_epyc1/synthetic1/mkl_ie_d.csv')
# convert_it('amd_epyc1/synthetic1/csr5_d.csv')
# convert_it('amd_epyc1/synthetic1/merge_d.csv')
# convert_it('amd_epyc1/synthetic1/aocl_optmv_d.csv')
# convert_it('amd_epyc1/syntbhetic1/csr_naive_d.csv')
# convert_it('amd_epyc1/synthetic1/sparsex_d.csv')
# convert_it('amd_epyc1/synthetic1/sell_C_s_d.csv')

---

In [ ]:
with open('features_small2.csv') as f:
    features_small2 = f.readlines()
for i in range(len(features_small2)):
    features_small2[i] = features_small2[i].strip('\n')
# print(features_small[0])
# print(features_small2[0])

prefix = 'amd_hawk'

impl_list         = ['csr_naive',     'csr5',  'mkl_ie']
impl_list2        = ['Naive_CSR_CPU', 'CSR5', 'MKL_IE']
folder_ids        = [1]

for folder_id in folder_ids:
    for impl,impl2 in zip(impl_list, impl_list2):
        do_it(folder_id, impl, impl2, features_small2, prefix)

concat_it(folder_ids, impl_list, impl_list2, prefix)

In [ ]:
prefix = 'amd_epyc1_old'

impl_list         = ['aocl_optmv', 'csr5', 'csr_naive',     'csr_x86_vector',    'csr_x86_vector_perfect_nnz_balance', 'merge', 'mkl_ie', 'sparsex', 'sell_C_s' ]
impl_list2        = ['AOCL_OPTMV', 'CSR5', 'Naive_CSR_CPU', 'Custom_CSR_BV_x86', 'Custom_CSR_PBV_x86',                 'MERGE', 'MKL_IE', 'SparseX', 'SELL-32-1']
folder_ids        = [1,2,3,4,5]

for folder_id in folder_ids:
    for impl,impl2 in zip(impl_list, impl_list2):
        # do_it(folder_id, impl, impl2, features, prefix)
        do_it(folder_id, impl, impl2, features_small, prefix)

concat_it(folder_ids, impl_list, impl_list2, prefix)

In [ ]:
prefix = 'arm'

impl_list         = ['armpl', 'csr_naive',     'merge', 'sparsex', 'sell_C_s' ]
impl_list2        = ['ARMPL', 'Naive_CSR_CPU', 'MERGE', 'SparseX', 'SELL-32-1']
folder_ids        = [1,2,3,4,5]

for folder_id in folder_ids:
    for impl,impl2 in zip(impl_list, impl_list2):
        # do_it(folder_id, impl, impl2, features, prefix)
        do_it(folder_id, impl, impl2, features_small, prefix)

concat_it(folder_ids, impl_list, impl_list2, prefix)

In [7]:
prefix = 'epyc5-A100'

impl_list         = ['coo', 'csr',     'merge', 'sell']
impl_list2        = ['cuSPARSE_coo11', 'cuSPARSE_csr11', 'Merge_11', 'SELL-32-1']
folder_ids        = [1,2,3,4,5]

for folder_id in folder_ids:
    for impl,impl2 in zip(impl_list, impl_list2):
        do_it(folder_id, impl, impl2, features, prefix)
        # do_it(folder_id, impl, impl2, features_small, prefix, is_gpu=True)

concat_it(folder_ids, impl_list, impl_list2, prefix, is_gpu=True)

epyc5-A100/synthetic1/coo_d.csv does not exist
epyc5-A100/synthetic1/csr_d.csv does not exist
epyc5-A100/synthetic1/merge_d.csv does not exist
epyc5-A100/synthetic1/sell_d.csv does not exist
epyc5-A100/synthetic2/coo_d.csv does not exist
epyc5-A100/synthetic2/csr_d.csv does not exist
epyc5-A100/synthetic2/merge_d.csv does not exist
epyc5-A100/synthetic2/sell_d.csv does not exist
epyc5-A100/synthetic3/coo_d.csv does not exist
epyc5-A100/synthetic3/csr_d.csv does not exist
epyc5-A100/synthetic3/merge_d.csv does not exist
epyc5-A100/synthetic3/sell_d.csv does not exist
epyc5-A100/synthetic4/coo_d.csv does not exist
epyc5-A100/synthetic4/csr_d.csv does not exist
epyc5-A100/synthetic4/merge_d.csv does not exist
epyc5-A100/synthetic4/sell_d.csv does not exist
epyc5-A100/synthetic5/coo_d.csv does not exist
epyc5-A100/synthetic5/csr_d.csv does not exist
epyc5-A100/synthetic5/merge_d.csv does not exist
epyc5-A100/synthetic5/sell_d.csv does not exist
